In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 

/dshome/WoongLab/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_train_data.csv')

In [3]:
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_test_data.csv')

In [4]:
train_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,27.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,2,2.0,1.0,0.0,0.0,13.0,4,1,0.0
1,1.0,1.0,1.0,25.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,0.0,10.0,5,6,0.0
2,0.0,0.0,1.0,24.0,1.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,7.0,3,6,0.0
3,1.0,1.0,1.0,24.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,2,0.0,30.0,1.0,0.0,11.0,3,3,0.0
4,0.0,0.0,1.0,20.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,1,15.0,0.0,0.0,0.0,2.0,5,5,0.0


In [5]:
test_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,30.0,1.0,1.0,0,0.0,0.0,1.0,...,0.0,1,0.0,0.0,0.0,1.0,11.0,5,7,0.0
1,1.0,1.0,1.0,27.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,1,4.0,0.0,0.0,0.0,11.0,5,5,0.0
2,1.0,0.0,1.0,31.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,2,3.0,3.0,0.0,0.0,6.0,5,6,0.0
3,0.0,1.0,1.0,26.0,1.0,0.0,0,1.0,0.0,0.0,...,0.0,2,0.0,0.0,0.0,0.0,12.0,5,5,0.0
4,1.0,1.0,1.0,25.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,9.0,4,6,0.0


In [6]:
train_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker','GenHlth','Age','Stroke','HighBP','DiffWalk','HighChol','PhysHlth','Sex','Diabetes','BMI','Income']

In [7]:
test_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker']

In [8]:
train_X=train_data.loc[:,train_stage_features] 
train_y=train_data['HeartDiseaseorAttack']
new_test_data_X=test_data[test_stage_features]
new_test_data_y=test_data['HeartDiseaseorAttack']

In [9]:
train_y.value_counts()

0.0    183830
1.0     19114
Name: HeartDiseaseorAttack, dtype: int64

In [10]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [11]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-13 04:27:56,511] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_116527/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-13 04:28:13,008] Trial 0 finished with value: 0.8436413178583276 and parameters: {'learning_rate': 0.04249473018154181, 'max_depth': 8, 'n_estimators': 951}. Best is trial 0 with value: 0.8436413178583276.
/tmp/ipykernel_116527/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-13 04:28:19,449] Trial 1 finished w

Best hyperparameters: {'learning_rate': 0.023254513760080378, 'max_depth': 3, 'n_estimators': 620}
Best AUC: 0.8500


In [12]:
lgb=lgb.LGBMClassifier(learning_rate= 0.023254513760080378,max_depth=3,n_estimators=620,scale_pos_weight=ratio,random_state=42)

In [13]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.023254513760080378, max_depth=3,
               n_estimators=620, random_state=42,
               scale_pos_weight=9.617557811028565)

In [15]:
train_prob=pd.DataFrame()
train_prob['prob']=lgb.predict_proba(train_X)[:,1]

In [16]:
train_prob['1_prob']=1-train_prob['prob']
train_prob['odds']=train_prob['prob']/train_prob['1_prob']

In [17]:
train_prob['loogits']=np.log(train_prob['odds'])
train_X_new=train_X[test_stage_features]

In [18]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_prob['loogits'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-13 04:39:26,687] A new study created in memory with name: no-name-82b69bd0-ca66-41ba-9e8e-eafaf8d42bd0
/tmp/ipykernel_116527/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-13 04:39:38,754] Trial 0 finished with value: 2.337630902824071 and parameters: {'n_estimators': 720, 'max_depth': 7, 'learning_rate': 0.015159001682455028}. Best is trial 0 with value: 2.337630902824071.
/tmp/ipykernel_116527/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-1

{'n_estimators': 561, 'max_depth': 4, 'learning_rate': 0.02801874603051041}


In [19]:
# Print best hyperparameters and auc

print(f'Best hyperparameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.4f}')

Best hyperparameters: {'n_estimators': 561, 'max_depth': 4, 'learning_rate': 0.02801874603051041}
Best RMSE: 2.3366


In [20]:
model = LGBMRegressor(n_estimators=561,max_depth=4,learning_rate=  0.02801874603051041)

In [21]:
model.fit(train_X_new,train_prob['loogits'])

LGBMRegressor(learning_rate=0.02801874603051041, max_depth=4, n_estimators=561)

In [22]:
test_data=pd.DataFrame()
test_data['loogits']=model.predict(new_test_data_X)

In [23]:
test_data['new_prob']=1/(1+np.exp(-test_data['loogits']))

In [24]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.5,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [25]:
get_clf_prob(new_test_data_y,test_data['new_prob'])

임계값:  0.5
오차행렬
[[40923  5034]
 [ 3566  1213]]
정확도: 0.8305, 정밀도: 0.1942, 재현율: 0.2538, F1:0.5625, AUC: 0.6678
